In [ ]:
import os
import cv2
import numpy as np
import warnings
from sklearn.metrics import classification_report

warnings.filterwarnings('ignore')

# 특성을 추출하는 함수 정의(평균 색상)
def averagecolor(image):
    return np.mean(image, axis=(0, 1))

red_sum = []
blue_sum = []
white_sum = []

for i in os.listdir('train/red/'):
    red_img = cv2.imread('train/red/'+i)
    av_red = averagecolor(red_img)
    red_sum.append(av_red)

for i in os.listdir('train/blue/'):
    blue_img = cv2.imread('train/blue'+i)
    blue_sum.append(blue_img)

for i in os.listdir('train/white/'):
    white_img = cv2.imread('train/white'+i)
    blue_sum.append(white_img)

print(red_sum)

[array([40.48169699, 70.86323727, 94.16367496]), array([ 55.56985289,  98.61699967, 141.3171112 ]), array([134.3160841 , 126.95805941,  54.88823688]), array([ 46.92105632,  87.69864258, 127.73651693]), array([44.58400248, 56.06719747, 94.56438527]), array([21.29198407, 26.05636223, 63.92540012]), array([ 50.5623521 ,  23.03474545, 194.16749002]), array([ 27.93003864,  38.6965518 , 138.08672515]), array([ 54.9662573 ,  66.00824602, 148.40794755]), array([113.91518717, 126.58757812, 135.89076497]), array([20.21004931, 22.27889514, 62.69611667]), array([195.73856384, 199.02698997, 217.82970293]), array([203.8857489 , 210.16579529, 215.69136841]), array([ 37.60952057,  36.63254546, 133.23088541]), array([ 16.2558624 ,  14.20787276, 144.73047876]), array([ 1.29074074,  1.22671605, 19.3702321 ]), array([145.7205424 , 145.66846842, 209.09757568]), array([ 9.78618063, 20.31507356, 60.56314711]), array([ 81.49237179, 106.39477998, 132.58890022]), array([ 17.64310571,  31.1180054 , 119.27317515]

In [ ]:
realtestY = np.array(["blue"*30,
                     "red"*24,
                     "white"*29])
def evaluateaccuracy(filenames,predictedY):
    predictedY = np.array(predictedY)
    if (np.sum(realtestY!=predictedY)>0):
        print ("Wrong Predictions: (filename, labelled, predicted) ")
        print (np.dstack([filenames,realtestY,predictedY]).squeeze()[(realtestY!=predictedY)])
    # 전체 예측의 백분율로 일치하는 (정확한) 예측을 계산합니다.
    return "Correct :"+ str(np.sum(realtestY==predictedY)) + ". Wrong: "+str(np.sum(realtestY!=predictedY)) + ". Correctly Classified: " + str(np.sum(realtestY==predictedY)*100/len(predictedY))+"%"

In [19]:
import os
import numpy as np
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 경로 설정
train_dir = 'train'
test_dir = 'test'

# 평균 RGB 값을 구하는 함수
def calculate_mean_rgb(image_path):
    image = Image.open(image_path).convert('RGB')  # RGB 형식으로 변환
    np_image = np.array(image)
    mean_rgb = np.mean(np_image, axis=(0, 1))  # 이미지의 평균 RGB 계산
    return mean_rgb

# 학습 데이터를 로드하고 평균 RGB를 계산
train_data = []
train_labels = []

# 각 색상별 폴더에 접근하여 평균 RGB 값을 구하고 레이블 추가
for label, color in enumerate(['red', 'blue', 'white']):  # 0:red, 1:blue, 2:white
    color_dir = os.path.join(train_dir, color)
    for file_name in os.listdir(color_dir):
        file_path = os.path.join(color_dir, file_name)
        if file_path.endswith(('.jpg', '.png')):  # 이미지 파일만 처리
            mean_rgb = calculate_mean_rgb(file_path)
            train_data.append(mean_rgb)
            train_labels.append(label)

train_data = np.array(train_data)
train_labels = np.array(train_labels)

# 학습 데이터 분할 (80% 학습, 20% 검증)
X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# KNN, SVM, 랜덤 포레스트 모델 초기화
knn = KNeighborsClassifier(n_neighbors=5)
svm = SVC(kernel='linear')
rf = RandomForestClassifier(n_estimators=100)

# 모델 훈련
knn.fit(X_train, y_train)
svm.fit(X_train, y_train)
rf.fit(X_train, y_train)

# 검증 데이터에 대한 정확도 평가
knn_accuracy = accuracy_score(y_val, knn.predict(X_val))
svm_accuracy = accuracy_score(y_val, svm.predict(X_val))
rf_accuracy = accuracy_score(y_val, rf.predict(X_val))

print(f"KNN Validation Accuracy: {knn_accuracy * 100:.2f}%")
print(f"SVM Validation Accuracy: {svm_accuracy * 100:.2f}%")
print(f"Random Forest Validation Accuracy: {rf_accuracy * 100:.2f}%")

# 테스트 데이터 로드 및 평균 RGB 계산
test_data = []
test_file_names = []

for file_name in os.listdir(test_dir):
    file_path = os.path.join(test_dir, file_name)
    if file_path.endswith(('.jpg', '.png')):  # 이미지 파일만 처리
        mean_rgb = calculate_mean_rgb(file_path)
        test_data.append(mean_rgb)
        test_file_names.append(file_name)

test_data = np.array(test_data)

# 테스트 데이터 예측 수행
knn_predictions = knn.predict(test_data)
svm_predictions = svm.predict(test_data)
rf_predictions = rf.predict(test_data)

# 테스트 데이터 예측 결과 출력
print("\nKNN Test Predictions:")
for file_name, pred in zip(test_file_names, knn_predictions):
    print(f"{file_name}: {['Red', 'Blue', 'White'][pred]}")

print("\nSVM Test Predictions:")
for file_name, pred in zip(test_file_names, svm_predictions):
    print(f"{file_name}: {['Red', 'Blue', 'White'][pred]}")

print("\nRandom Forest Test Predictions:")
for file_name, pred in zip(test_file_names, rf_predictions):
    print(f"{file_name}: {['Red', 'Blue', 'White'][pred]}")

KNN Validation Accuracy: 84.21%
SVM Validation Accuracy: 87.72%
Random Forest Validation Accuracy: 87.72%

KNN Test Predictions:
Image_1.jpg: Blue
Image_10.jpg: Blue
Image_11.jpg: Blue
Image_12.jpg: Blue
Image_13.jpg: Blue
Image_14.jpg: Blue
Image_15.png: Blue
Image_16.png: Blue
Image_17.jpg: Blue
Image_18.jpg: Blue
Image_19.jpg: Blue
Image_2.jpg: Blue
Image_20.png: Blue
Image_21.jpg: Blue
Image_22.jpg: Blue
Image_23.jpg: Blue
Image_24.jpg: Blue
Image_25.jpg: Blue
Image_26.png: Blue
Image_27.jpg: Blue
Image_28.jpg: Blue
Image_29.png: Red
Image_3.jpg: Blue
Image_30.png: Blue
Image_4.jpg: Blue
Image_5.jpg: Blue
Image_6.jpg: Blue
Image_7.jpg: Blue
Image_8.jpg: Blue
Image_9.jpg: Blue
red (1).jpg: Red
red (10).jpg: Red
red (11).jpg: Red
red (12).jpg: Red
red (13).jpg: Red
red (14).jpg: Red
red (15).jpg: Red
red (16).jpg: White
red (17).jpg: Red
red (18).jpg: Red
red (19).jpg: Red
red (2).jpg: Red
red (20).jpg: Red
red (21).jpg: Red
red (22).jpg: Red
red (23).jpg: Red
red (24).jpg: Red
red (

In [20]:
import os
import numpy as np
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 경로 설정
train_dir = 'train'
test_dir = 'test'

# 평균 RGB 값을 구하는 함수
def calculate_mean_rgb(image_path, resize=(50, 50)):
    try:
        with Image.open(image_path) as img:
            img = img.convert('RGB')
            img = img.resize(resize)  # 이미지 크기 축소하여 계산 최적화
            np_image = np.array(img)
            mean_rgb = np.mean(np_image, axis=(0, 1))  # 평균 RGB 계산
            return mean_rgb
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None  # 오류 발생 시 None 반환

# 학습 데이터를 로드하고 평균 RGB를 계산
train_data = []
train_labels = []

# 각 색상별 폴더에 접근하여 평균 RGB 값을 구하고 레이블 추가
for label, color in enumerate(['red', 'blue', 'white']):  # 0:red, 1:blue, 2:white
    color_dir = os.path.join(train_dir, color)
    if os.path.exists(color_dir):
        for file_name in os.listdir(color_dir):
            file_path = os.path.join(color_dir, file_name)
            if file_path.endswith(('.jpg', '.png')):  # 이미지 파일만 처리
                mean_rgb = calculate_mean_rgb(file_path)
                if mean_rgb is not None:
                    train_data.append(mean_rgb)
                    train_labels.append(label)

train_data = np.array(train_data)
train_labels = np.array(train_labels)

# 학습 데이터 분할 (80% 학습, 20% 검증)
X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# 모델 초기화 및 학습
models = {
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(kernel='linear'),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}

# 각 모델에 대해 학습, 검증 정확도 계산
for model_name, model in models.items():
    model.fit(X_train, y_train)
    val_predictions = model.predict(X_val)
    accuracy = accuracy_score(y_val, val_predictions)
    print(f"{model_name} Validation Accuracy: {accuracy * 100:.2f}%")

# 테스트 데이터 로드 및 평균 RGB 계산
test_data = []
test_file_names = []

for file_name in os.listdir(test_dir):
    file_path = os.path.join(test_dir, file_name)
    if file_path.endswith(('.jpg', '.png')):  # 이미지 파일만 처리
        mean_rgb = calculate_mean_rgb(file_path)
        if mean_rgb is not None:
            test_data.append(mean_rgb)
            test_file_names.append(file_name)

test_data = np.array(test_data)

# 테스트 데이터 예측 및 결과 출력
for model_name, model in models.items():
    predictions = model.predict(test_data)
    print(f"\n{model_name} Test Predictions:")
    for file_name, pred in zip(test_file_names, predictions):
        print(f"{file_name}: {['Red', 'Blue', 'White'][pred]}")

KNN Validation Accuracy: 84.21%
SVM Validation Accuracy: 85.96%
Random Forest Validation Accuracy: 89.47%

KNN Test Predictions:
Image_1.jpg: Blue
Image_10.jpg: Blue
Image_11.jpg: Blue
Image_12.jpg: Blue
Image_13.jpg: Blue
Image_14.jpg: Blue
Image_15.png: Blue
Image_16.png: Blue
Image_17.jpg: Blue
Image_18.jpg: Blue
Image_19.jpg: Blue
Image_2.jpg: Blue
Image_20.png: Blue
Image_21.jpg: Blue
Image_22.jpg: Blue
Image_23.jpg: Blue
Image_24.jpg: Blue
Image_25.jpg: Blue
Image_26.png: Blue
Image_27.jpg: Blue
Image_28.jpg: Blue
Image_29.png: Red
Image_3.jpg: Blue
Image_30.png: Blue
Image_4.jpg: Blue
Image_5.jpg: Blue
Image_6.jpg: Blue
Image_7.jpg: Blue
Image_8.jpg: Blue
Image_9.jpg: Blue
red (1).jpg: Red
red (10).jpg: Red
red (11).jpg: Red
red (12).jpg: Red
red (13).jpg: Red
red (14).jpg: Red
red (15).jpg: Red
red (16).jpg: White
red (17).jpg: Red
red (18).jpg: Red
red (19).jpg: Red
red (2).jpg: Red
red (20).jpg: Red
red (21).jpg: Red
red (22).jpg: Red
red (23).jpg: Red
red (24).jpg: Red
red (